# Data Assimilation (DA) Concepts through Reanalysis Practical

This practical introduces fundamental DA concepts using the ERA5 and NCEP reanalysis datasets, which are products of the latest techniques.

In this practical, three out of four tasks use ERA5. ERA5 is produced by ECMWF using an advanced 4D-Var assimilation system with a 10-member ensemble.

While ERA5 serves as an excellent reference point for concepts used in DA, it is worth noting that our practical is intentionally simplified compared to operational DA systems. For example, in this practical, the observations are artificially generated from the reanalysis, and the DA algorithm, which demonstrates core principles, is simplified.

In Tasks 1-3, we'll focus on the North Atlantic/UK region, examining 500hPa geopotential height patterns. Here, we provide ERA5 reanalysis for 1 March 2025, including simulated ensemble members, ensemble mean, ensemble spread, and reanalysis from ERA5 data. Through these exercises, you'll discover how DA creates "maps without gaps" by blending model information with observations, and how uncertainty is represented in ensemble systems.

In Task 4, we will explore how changes in observing networks can impact analysis products over time through a comparison between ERA5 and NCEP reanalysis datasets.

First, let us set up the data used in this practical.

In [ ]:
%%capture
!pip install cartopy

In [ ]:
import os
import shutil
import kagglehub
# get necessary dependency scripts
!git clone https://github.com/darc-reading/DA-training-2025/
%cd DA-training-2025
# get ERA5 and NCEP datasets
da_practice_data_path = kagglehub.dataset_download('sukuncheng/da-practice-data')
DATA_DIR = os.path.join('working', 'da_practical_data') # path of working directory
os.makedirs(DATA_DIR, exist_ok=True) # create working directory
shutil.copytree(os.path.join(da_practice_data_path, 'da_practice_data'), DATA_DIR,  dirs_exist_ok=True)
 # copy the file to working directory

Here, we copy the sample data to a working directory for our manipulation.

## Task 1: Introduce to ERA5 as a DA Product(5 mins)

In this task, we will visualise the 500hPa geopotential height from ERA5 reanalysis on 1 March, 2025 over the North Atlantic/UK region. This should give you a sense of the atmospheric circulation. A high pressure system sitting in the Atlantic and a low pressure system in France.


## Visualise 500hPa geopotential height from ERA5 reanalysis

Definition: Geopotential height is a vertical coordinate that represents the height of a pressure level in the atmosphere, corrected for variations in gravity. It is commonly used on weather maps and in numerical weather prediction.

On a 500 hPa weather map:
- The "height" of the 500 hPa pressure level might be 5,500 meters in geopotential height.
- This means the air pressure drops to 500 hPa at approximately that height, adjusted for Earth’s gravity.

In [ ]:
import reanalysis_task_1
reanalysis_task_1.plot_500_geopotential(DATA_DIR)

# Task 2: A simplified DA experiment (15 mins)

In DA, the update equation of the analysis can be:
$$\mathbf{x}^a = \mathbf{x}^b + \mathbf{K}\mathbf{d}$$
where:
- $\mathbf{x}^a$ is the analysis state
- $\mathbf{x}^b$ is the background (model) state
- $\mathbf{d} = \mathbf{y} - \mathbf{H}(\mathbf{x}^b)$ is the innovation with $\mathbf{H}$ an observation operator that transform state vector to a vector expressed in the form of observations.

- $\mathbf{K}$ is the gain matrix. In a Kalman filter, $\mathbf{K} = \mathbf{B}\mathbf{H}^\mathrm{T}(\mathbf{H}\mathbf{B}\mathbf{H}^\mathrm{T} + \mathbf{R})^{-1}$ where $\mathbf{B}$ is the background error covariance, $\mathbf{R}$ is the observation error covariance. This highlights that the analysis is obtained by balancing the error covariance matrix (uncertainty) between observations and background.

In this task, we assume that the ERA5 reanalysis is the true 500hPa geopotential height $\mathbf{x}^t$ on 1 March 2025. We now artifically generate the background from the truth:
$$\mathbf{x}^b = \mathbf{x}^t + \mathbf{\epsilon}^b$$
where $\mathbf{\epsilon}^b$ follows a Gaussian distribution with zero mean and a covariance matrix of $\mathbf{B}$,i.e., $\mathbf{\epsilon}^b \sim \mathscr{N}(\mathbf{0}, \mathbf{B})$.

For the sake of simplicity, we use a spatial **correlation function** to build the $\mathbf{B}$ matrix, which depends on a given spatial correlation length scale, $L$. This Gaussian correlation decays exponentially with distance:
$$B(r'_{i,j},L) = \exp\left(-\frac{1}{2}\left(\frac{r'_{i,j}}{L}\right)^2\right)$$
with $r'_{i,j}$ the distance between the grid points at the $i$-th and $j$-th elements of the state vector. The full background error covariance matrix $\mathbf{B}$ would then be formed by multiplying this correlation matrix by the background error variances $\sigma^2_b$.



Here, $L$ controls how spatially correlated the background errors are assumed to be.
- $L$ defines the structure of the background error covariance matrix $\mathbf{B}$
- Larger values of $L$ spread observation information further spatially
- When too large $L$, observations can introduce unrealistic long-range relationships
- When too small $L$, observations have minimal spatial impact

To illustrate these points, we can visualise the covariance using the above equation with different $L$ between the centre of the grid point and the rest of the domain.

In [ ]:
import reanalysis_task_2
reanalysis_task_2.plot_correlation(DATA_DIR)

In this practice, the observation operator $\mathbf{H}$ is simplified to only randomly selects a subset of total grid points. Like many DA systems, we assume that observations are made independently so that observation errors are not correlated.
The observations are generated synthetically from the background field by adding random errors
$$\mathbf{y} = \mathbf{H}\mathbf{x}^b + \mathbf{\epsilon}$$
where $\mathbf{\epsilon}$ follows a multivariate Gaussian distribution with zero mean and covariance matrix $\mathbf{R} = \sigma^2_o \mathbf{I}$, where $\sigma^2_o$ represents the observation error variance, i.e., $\mathbf{\epsilon} \sim \mathscr{N}(\mathbf{0}, \mathbf{R})$.


Because the DA scheme weights the observation and background information based on their error variances:
- Higher values of $\sigma^2_o$ give less weight to observations
- Critical when observations contain significant instrumental errors or representativeness issues

In this setup, observations can be assimilated serially. This means that, for a total of $p$ number of observations, at each time, only one observation value can be assimilated. This approach can simplify the analysis equation ($\mathbf{x}^a = \mathbf{x}^b + \mathbf{K}\mathbf{d}$):
$$\mathbf{x}^a_{k+1} = \mathbf{x}^a_k + \left(\mathbf{\rho}_k(r',r) \circ \mathbf{b}_k(r',L)\right) \cdot \gamma_k \cdot d_k$$
where $k = 1, \dots, p$. When $k=1$, $\mathbf{x}^a_k = \mathbf{x}^b$ is the initial background state. The innovation becomes a scalar value $d_k = y_k - \mathbf{H}_k\mathbf{x}^a_k$ with $\mathbf{H}_k$ selecting the background grid points corresponding to the $k$-th observation.

The scalar Kalman gain is given by:
 $$\gamma_k = \frac{1}{1 + \sigma^2_o}$$
which corresponds to simplified version of $(\mathbf{H}\mathbf{B}\mathbf{H}^\mathrm{T} + \mathbf{R})^{-1}$, assuming unit background error variance and observation error variance of $\sigma^2_o$.

The term, $\mathbf{\rho}_k(r',r) \circ \mathbf{b}_k(r',L)$, is a vector as a simplfied version of $\mathbf{B}\mathbf{H}^\mathrm{T}$ with $\circ$ an element-wise product and $r$ the localisation radius. The term, $\mathbf{b}_k(r',L)$ is a row/column of the full background error covariance matrix $\mathbf{B}$ that represents the background error covariances between observation location and all grid points:
$$\mathbf{b}_k(r',L) = \exp\left(-\frac{1}{2}\left(\frac{r'}{L}\right)^2\right)$$
and the localisation term,
$$\mathbf{\rho}_k(r',r) = \max\left(0, 1 - \frac{r'}{r}\right)$$
where $r'$ is the distance between each grid point and the observation point. This term makes the decay of spatial correlation faster which effectively limits the influence of each observation spatially. Localisation is typically used in ensemble DA systems to address sampling error. Although we do not use an ensemble DA system, localisation still show its effect spatially on the observations influence.

Now let's explore background error correlation length scale $L$, localisation radius $r$, and observation error $\sigma^2_o$ affect the analysis result.

In [ ]:
import reanalysis_task_2
reanalysis_task_2.interactive_da_params(DATA_DIR)

### Discussion:
1. The background error correlation scale ${L}$ and observation error weight $\sigma^2_o$ affect which information source (model or observations) dominates in different regions. When you change these parameters, what do you see from the plots?

2. In regions with dense/spare observations, how does the analysis differ from the background?

3. How does increased localization radius $r$ affect the spatial structure of the analysis increments? Why could an inappropriately large radius lead to physically unrealistic analysis fields?



# Task 3: Uncertainty Representation via Ensemble Spread (20 mins)
ERA5 includes a 10-member ensemble of DA cycles that provides valuable uncertainty information.
The uncertainty is typically represented by the ensemble spread. With an ensemble, the background error covariance matrix could be approximated.

Figures below show the spatial distribution of 500 hPa geopotential heights decameters: ensemble mean (left) and spread (right).

In [ ]:
import reanalysis_task_3
ds = reanalysis_task_3.get_data(DATA_DIR)
reanalysis_task_3.plot_ensemble(ds)

### Interactive Ensemble Size Emulation
Observe differences by tuning ensemble size

In [ ]:
# Run the interactive ensemble size emulation
import reanalysis_task_3
ds = reanalysis_task_3.get_data(DATA_DIR)
# changing the ensemble size for any integer value from 1 to 10
reanalysis_task_3.changing_ensemble_size(ds)

### Discussion:

1. How could ensemble be related to the error covariance matrix?

2. What could be the possible reasons that ocean regions typically showing different uncertainty patterns than land regions?

3. What happens to the spread estimate as you reduce the ensemble size?

# Task 4: Observing System Impact on Reanalysis Time Series (20 mins)

In Task 4, we will compare ERA5 and NCEP Reanalyses in the Southern Ocean. The Southern Ocean remains one of the most observation-sparse regions on Earth, making atmospheric reanalysis products essential research tools for understanding climate patterns and variability.

This analysis examines how the introduction of Advanced TIROS Operational Vertical Sounder (ATOVS) satellite data in 1998 influenced Total Column Water Vapor (TCWV) estimates in two major reanalysis products.

Let's examine how the introduction of new satellite observations affects the ERA5/NCEP reanalysis:

- ERA5 monthly means provide averaged atmospheric, land, and ocean-wave variables aggregated from the hourly ERA5 reanalysis data, a high spatial resolution (0.25° × 0.25°).

- NCEP/NCAR dataset offers a lower spatial resolution (2.5° × 2.5°) but provides a longer temporal coverage, making it useful for long-term climate studies and for validating the ERA5 reanalysis.



## Visualize the time series of ERA5/NCEP spatial averages

In [ ]:
import reanalysis_task_4
reanalysis_task_4.plot_timeseries(DATA_DIR)

### Discussion

Looking at ERA5/NCEP time series from the Southern Ocean (10S - 50S), there are indeed observable impacts from the ATOVS satellite introduction in 1998.
- Shift in mean values response to ATOVS: Both reanalysis products show similar responses to the ATOVS introduction - a decrease in mean TCWV values (ERA5 by 0.32 kg/m² and NCEP by 0.33 kg/m²). This shift is typical when a reanalysis system incorporates new satellite data that provides more accurate measurements.

- It suggests that, in both systems, the assimilation of new satellite data leads to slightly drier conditions than previously estimated in the Southern Ocean region.

- The seasonal cycle remains consistent before and after ATOVS, which reflects both reanalysis systems working to maintain temporal consistency while incorporating new data.

- While both reanalyses show decreased variability immediately following ATOVS introduction, ERA5 exhibits more pronounced variability peaks during 2000-2001 and 2003-2004.

- NCEP reanalysis shows a smoother variability pattern than ERA5 after 2000. This divergence in behavior is mainly due that ERA5's more complex assimilation system more dynamic response to other new sateliate observations.






# Wrap-up & Discussion (5 mins)
In this practical, we've explored several key aspects of DA as implemented in operational reanalysis:

- We explored how DA obtain an analysis from the model and observations based on their respective uncertainties.

- We investigated how key DA parameters such as localisation radius and error correlation lengthscale affect the analysis result.

- We examined how an ensemble can provide information about analysis uncertainty and how ensemble size affects these estimates.

- We identified a change in reanalysis when new satellite data is introduced, highlighting challenges for trend analysis.